# Caminho 1D
Para o caminho 1D, usaremos cores contrastantes para as linhas.

In [ ]:
import numpy as np
import plotly.express as px


def monte_carlo_1d(iterations, path_length):
    best_path = None
    best_score = float("inf")

    for _ in range(iterations):
        path = np.sin(np.linspace(0, 4 * np.pi, path_length)) + 0.1 * np.random.randn(
            path_length
        )
        score = np.sum(np.abs(path))  # Simple scoring function

        if score < best_score:
            best_score = score
            best_path = path

    return best_path, best_score


# Encontrar o melhor caminho
path_length = 100
iterations = 1000
original_path_1d = np.sin(np.linspace(0, 4 * np.pi, path_length))
best_path_1d, best_score_1d = monte_carlo_1d(iterations, path_length)

fig_1d = px.line(x=np.arange(path_length), y=original_path_1d, title="Best Path in 1D")
fig_1d.add_scatter(
    x=np.arange(path_length),
    y=best_path_1d,
    mode="lines",
    name="Monte Carlo Path",
    line=dict(color="red"),
)
fig_1d.data[0].update(name="Original Path", line=dict(color="blue"))
fig_1d.show()


# Caminho 2D
Para o caminho 2D, usaremos cores contrastantes para as linhas.

In [ ]:
import plotly.graph_objs as go


def monte_carlo_2d(iterations, path_length):
    best_path = None
    best_score = float("inf")

    for _ in range(iterations):
        t = np.linspace(0, 4 * np.pi, path_length)
        path = np.vstack((np.sin(t), np.cos(2 * t))).T + 0.1 * np.random.randn(
            path_length, 2
        )
        score = np.sum(
            np.linalg.norm(np.diff(path, axis=0), axis=1)
        )  # Euclidean distance

        if score < best_score:
            best_score = score
            best_path = path

    return best_path, best_score


# Encontrar o melhor caminho
path_length = 100
iterations = 1000
t = np.linspace(0, 4 * np.pi, path_length)
original_path_2d = np.vstack((np.sin(t), np.cos(2 * t))).T
best_path_2d, best_score_2d = monte_carlo_2d(iterations, path_length)

fig_2d = go.Figure()
fig_2d.add_trace(
    go.Scatter(
        x=original_path_2d[:, 0],
        y=original_path_2d[:, 1],
        mode="lines",
        name="Original Path",
        line=dict(color="blue"),
    )
)
fig_2d.add_trace(
    go.Scatter(
        x=best_path_2d[:, 0],
        y=best_path_2d[:, 1],
        mode="lines",
        name="Monte Carlo Path",
        line=dict(color="red"),
    )
)
fig_2d.update_layout(title="Best Path in 2D", xaxis_title="X", yaxis_title="Y")
fig_2d.show()


# Caminho 3D
Para o caminho 3D, usaremos cores contrastantes para as linhas.

In [ ]:
def monte_carlo_3d(iterations, path_length):
    best_path = None
    best_score = float("inf")

    for _ in range(iterations):
        t = np.linspace(0, 4 * np.pi, path_length)
        path = np.vstack((np.sin(t), np.cos(t), t)).T + 0.1 * np.random.randn(
            path_length, 3
        )
        score = np.sum(
            np.linalg.norm(np.diff(path, axis=0), axis=1)
        )  # Euclidean distance

        if score < best_score:
            best_score = score
            best_path = path

    return best_path, best_score


# Encontrar o melhor caminho
path_length = 100
iterations = 1000
t = np.linspace(0, 4 * np.pi, path_length)
original_path_3d = np.vstack((np.sin(t), np.cos(t), t)).T
best_path_3d, best_score_3d = monte_carlo_3d(iterations, path_length)

fig_3d = go.Figure()
fig_3d.add_trace(
    go.Scatter3d(
        x=original_path_3d[:, 0],
        y=original_path_3d[:, 1],
        z=original_path_3d[:, 2],
        mode="lines",
        name="Original Path",
        line=dict(color="blue"),
    )
)
fig_3d.add_trace(
    go.Scatter3d(
        x=best_path_3d[:, 0],
        y=best_path_3d[:, 1],
        z=best_path_3d[:, 2],
        mode="lines",
        name="Monte Carlo Path",
        line=dict(color="red"),
    )
)
fig_3d.update_layout(
    title="Best Path in 3D",
    scene=dict(xaxis_title="X", yaxis_title="Y", zaxis_title="Z"),
)
fig_3d.show()


# Caminho 4D
Para o caminho 4D, usaremos cores contrastantes para as linhas e diferentes projeções.

In [ ]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots


def monte_carlo_4d(iterations, path_length):
    best_path = None
    best_score = float("inf")

    for _ in range(iterations):
        t = np.linspace(0, 4 * np.pi, path_length)
        path = np.vstack(
            (np.sin(t), np.cos(t), np.sin(2 * t), np.cos(2 * t))
        ).T + 0.1 * np.random.randn(path_length, 4)
        score = np.sum(
            np.linalg.norm(np.diff(path, axis=0), axis=1)
        )  # Euclidean distance

        if score < best_score:
            best_score = score
            best_path = path

    return best_path, best_score


# Encontrar o melhor caminho
path_length = 100
iterations = 1000
t = np.linspace(0, 4 * np.pi, path_length)
original_path_4d = np.vstack((np.sin(t), np.cos(t), np.sin(2 * t), np.cos(2 * t))).T
best_path_4d, best_score_4d = monte_carlo_4d(iterations, path_length)

# Create a subplot figure with 2x2 layout
fig = make_subplots(
    rows=2,
    cols=2,
    specs=[
        [{"type": "scatter3d"}, {"type": "scatter3d"}],
        [{"type": "scatter3d"}, {"type": "scatter3d"}],
    ],
    subplot_titles=[
        "Projection on X, Y, Z (Color: W)",
        "Projection on X, Y, W (Color: Z)",
        "Projection on X, Z, W (Color: Y)",
        "Projection on Y, Z, W (Color: X)",
    ],
)

# Adding traces for different projections
fig.add_trace(
    go.Scatter3d(
        x=original_path_4d[:, 0],
        y=original_path_4d[:, 1],
        z=original_path_4d[:, 2],
        mode="lines",
        line=dict(color="blue"),
        name="Original Path",
    ),
    row=1,
    col=1,
)

fig.add_trace(
    go.Scatter3d(
        x=best_path_4d[:, 0],
        y=best_path_4d[:, 1],
        z=best_path_4d[:, 2],
        mode="lines",
        line=dict(color="red"),
        name="Monte Carlo Path",
    ),
    row=1,
    col=1,
)

fig.add_trace(
    go.Scatter3d(
        x=original_path_4d[:, 0],
        y=original_path_4d[:, 1],
        z=original_path_4d[:, 3],
        mode="lines",
        line=dict(color="blue"),
        name="Original Path",
    ),
    row=1,
    col=2,
)

fig.add_trace(
    go.Scatter3d(
        x=best_path_4d[:, 0],
        y=best_path_4d[:, 1],
        z=best_path_4d[:, 3],
        mode="lines",
        line=dict(color="red"),
        name="Monte Carlo Path",
    ),
    row=1,
    col=2,
)

fig.add_trace(
    go.Scatter3d(
        x=original_path_4d[:, 0],
        y=original_path_4d[:, 2],
        z=original_path_4d[:, 3],
        mode="lines",
        line=dict(color="blue"),
        name="Original Path",
    ),
    row=2,
    col=1,
)

fig.add_trace(
    go.Scatter3d(
        x=best_path_4d[:, 0],
        y=best_path_4d[:, 2],
        z=best_path_4d[:, 3],
        mode="lines",
        line=dict(color="red"),
        name="Monte Carlo Path",
    ),
    row=2,
    col=1,
)

fig.add_trace(
    go.Scatter3d(
        x=original_path_4d[:, 1],
        y=original_path_4d[:, 2],
        z=original_path_4d[:, 3],
        mode="lines",
        line=dict(color="blue"),
        name="Original Path",
    ),
    row=2,
    col=2,
)

fig.add_trace(
    go.Scatter3d(
        x=best_path_4d[:, 1],
        y=best_path_4d[:, 2],
        z=best_path_4d[:, 3],
        mode="lines",
        line=dict(color="red"),
        name="Monte Carlo Path",
    ),
    row=2,
    col=2,
)

fig.update_layout(height=800, title="Best Path in 4D (Different Projections)")
fig.show()


# Implementação do Algoritmo de Monte Carlo em 4D Combinado
Vamos combinar todas as projeções em um único gráfico 4D e encontrar o melhor caminho considerando todas as dimensões.

In [ ]:
import numpy as np
import plotly.graph_objs as go


def monte_carlo_unified_4d(iterations, path_length):
    best_path = None
    best_score = float("inf")

    for _ in range(iterations):
        t = np.linspace(0, 4 * np.pi, path_length)
        path = np.vstack(
            (np.sin(t), np.cos(t), np.sin(2 * t), np.cos(2 * t))
        ).T + 0.1 * np.random.randn(path_length, 4)
        score = np.sum(
            np.linalg.norm(np.diff(path, axis=0), axis=1)
        )  # Euclidean distance

        if score < best_score:
            best_score = score
            best_path = path

    return best_path, best_score


# Encontrar o melhor caminho
path_length = 100
iterations = 1000
t = np.linspace(0, 4 * np.pi, path_length)
original_path_4d = np.vstack((np.sin(t), np.cos(t), np.sin(2 * t), np.cos(2 * t))).T
best_path_4d, best_score_4d = monte_carlo_unified_4d(iterations, path_length)

# Criar figura única com todas as projeções
fig = go.Figure()

# Adicionando traços para diferentes projeções
fig.add_trace(
    go.Scatter3d(
        x=original_path_4d[:, 0],
        y=original_path_4d[:, 1],
        z=original_path_4d[:, 2],
        mode="lines",
        line=dict(color="blue"),
        name="Original Path (X, Y, Z)",
    )
)

fig.add_trace(
    go.Scatter3d(
        x=best_path_4d[:, 0],
        y=best_path_4d[:, 1],
        z=best_path_4d[:, 2],
        mode="lines",
        line=dict(color="red"),
        name="Monte Carlo Path (X, Y, Z)",
    )
)

fig.add_trace(
    go.Scatter3d(
        x=original_path_4d[:, 0],
        y=original_path_4d[:, 1],
        z=original_path_4d[:, 3],
        mode="lines",
        line=dict(color="blue"),
        name="Original Path (X, Y, W)",
    )
)

fig.add_trace(
    go.Scatter3d(
        x=best_path_4d[:, 0],
        y=best_path_4d[:, 1],
        z=best_path_4d[:, 3],
        mode="lines",
        line=dict(color="red"),
        name="Monte Carlo Path (X, Y, W)",
    )
)

fig.add_trace(
    go.Scatter3d(
        x=original_path_4d[:, 0],
        y=original_path_4d[:, 2],
        z=original_path_4d[:, 3],
        mode="lines",
        line=dict(color="blue"),
        name="Original Path (X, Z, W)",
    )
)

fig.add_trace(
    go.Scatter3d(
        x=best_path_4d[:, 0],
        y=best_path_4d[:, 2],
        z=best_path_4d[:, 3],
        mode="lines",
        line=dict(color="red"),
        name="Monte Carlo Path (X, Z, W)",
    )
)

fig.add_trace(
    go.Scatter3d(
        x=original_path_4d[:, 1],
        y=original_path_4d[:, 2],
        z=original_path_4d[:, 3],
        mode="lines",
        line=dict(color="blue"),
        name="Original Path (Y, Z, W)",
    )
)

fig.add_trace(
    go.Scatter3d(
        x=best_path_4d[:, 1],
        y=best_path_4d[:, 2],
        z=best_path_4d[:, 3],
        mode="lines",
        line=dict(color="red"),
        name="Monte Carlo Path (Y, Z, W)",
    )
)

fig.update_layout(height=800, title="Best Unified Path in 4D")
fig.show()


# Utilizando 4D, porem com 3 espaciais e 1 temporal

In [ ]:
import numpy as np
import plotly.graph_objs as go

# Definir parâmetros
num_simulations = 100
time_steps = 50


# Função 4D a ser maximizada
def f_4d(x, y, z, t):
    return np.exp(-(x**2 + y**2 + z**2 + t**2) / 2)


# Função para gerar caminhos aleatórios em 4D
def generate_random_walk_4d(time_steps):
    path = np.cumsum(np.random.randn(time_steps, 4), axis=0)
    return path


# Simular vários caminhos
paths = [generate_random_walk_4d(time_steps) for _ in range(num_simulations)]

# Avaliar os caminhos e encontrar o melhor
best_score = -np.inf
best_path = None

for path in paths:
    score = np.sum(f_4d(path[:, 0], path[:, 1], path[:, 2], path[:, 3]))
    if score > best_score:
        best_score = score
        best_path = path

# Visualização do melhor caminho em 3D com Plotly (tempo como quarta dimensão)
fig = go.Figure()

# Adicionar todos os caminhos ao gráfico
for path in paths:
    fig.add_trace(
        go.Scatter3d(
            x=path[:, 0],
            y=path[:, 1],
            z=path[:, 2],
            mode="lines",
            line=dict(width=2),
            opacity=0.3,
        )
    )

# Adicionar o melhor caminho em destaque
fig.add_trace(
    go.Scatter3d(
        x=best_path[:, 0],
        y=best_path[:, 1],
        z=best_path[:, 2],
        mode="lines",
        line=dict(color="red", width=4),
        name="Best Path",
    )
)

# Adicionar ponto de origem
fig.add_trace(
    go.Scatter3d(
        x=[0],
        y=[0],
        z=[0],
        mode="markers",
        marker=dict(color="green", size=10),
        name="Origin",
    )
)

# Adicionar ponto de destino para o melhor caminho
fig.add_trace(
    go.Scatter3d(
        x=[best_path[-1, 0]],
        y=[best_path[-1, 1]],
        z=[best_path[-1, 2]],
        mode="markers",
        marker=dict(color="blue", size=10),
        name="Best Path Destination",
    )
)

# Configurações do layout do gráfico com tamanho ajustado
fig.update_layout(
    scene=dict(
        xaxis_title="X",
        yaxis_title="Y",
        zaxis_title="Z",
    ),
    title="Monte Carlo Paths in 3D Space with Time as 4th Dimension",
    showlegend=True,
    width=1200,  # Ajuste a largura do gráfico
    height=800,  # Ajuste a altura do gráfico
)

fig.show()


# Formas de representar as variações das 4D

In [ ]:
import numpy as np
import plotly.graph_objs as go


# Definir a função matemática em 4D
def f_4d(x, y, z, w):
    return np.sin(x) * np.cos(y) + np.sin(z) * np.cos(w)


# Algoritmo de Monte Carlo para encontrar o melhor caminho
def monte_carlo_4d(num_simulations, time_steps, dimension_interpretation):
    paths = []
    best_path = None
    best_value = float("inf")

    for _ in range(num_simulations):
        if dimension_interpretation == "4_spatial":
            path = np.cumsum(np.random.randn(time_steps, 4), axis=0)
        elif dimension_interpretation == "2_spatial_2_temporal":
            path = np.cumsum(np.random.randn(time_steps, 4), axis=0)
            path[:, 2] = np.arange(time_steps)  # t1 as sequential time steps
            path[:, 3] = np.arange(time_steps)  # t2 as sequential time steps
        elif dimension_interpretation == "3_temporal_1_spatial":
            path = np.cumsum(np.random.randn(time_steps, 4), axis=0)
            path[:, 0] = np.arange(time_steps)  # t1 as sequential time steps
            path[:, 1] = np.arange(time_steps)  # t2 as sequential time steps
            path[:, 2] = np.arange(time_steps)  # t3 as sequential time steps

        value = np.sum(f_4d(path[:, 0], path[:, 1], path[:, 2], path[:, 3]))

        if value < best_value:
            best_value = value
            best_path = path

        paths.append(path)

    return paths, best_path


# Visualização com Plotly para diferentes abordagens
def visualize_paths(paths, best_path, dimension_interpretation):
    fig = go.Figure()

    for path in paths:
        if dimension_interpretation == "4_spatial":
            fig.add_trace(
                go.Scatter3d(
                    x=path[:, 0],
                    y=path[:, 1],
                    z=path[:, 2],
                    mode="lines",
                    line=dict(width=2),
                    opacity=0.1,
                )
            )
        elif dimension_interpretation == "2_spatial_2_temporal":
            fig.add_trace(
                go.Scatter3d(
                    x=path[:, 0],
                    y=path[:, 1],
                    z=path[:, 2],
                    mode="lines",
                    line=dict(width=2),
                    opacity=0.1,
                )
            )
        elif dimension_interpretation == "3_temporal_1_spatial":
            fig.add_trace(
                go.Scatter3d(
                    x=path[:, 0],
                    y=path[:, 1],
                    z=path[:, 2],
                    mode="lines",
                    line=dict(width=2),
                    opacity=0.1,
                )
            )

    fig.add_trace(
        go.Scatter3d(
            x=best_path[:, 0],
            y=best_path[:, 1],
            z=best_path[:, 2],
            mode="lines",
            line=dict(color="red", width=4),
            name="Best Path",
        )
    )

    fig.add_trace(
        go.Scatter3d(
            x=[best_path[0, 0]],
            y=[best_path[0, 1]],
            z=[best_path[0, 2]],
            mode="markers",
            marker=dict(color="green", size=10),
            name="Origin",
        )
    )

    fig.add_trace(
        go.Scatter3d(
            x=[best_path[-1, 0]],
            y=[best_path[-1, 1]],
            z=[best_path[-1, 2]],
            mode="markers",
            marker=dict(color="blue", size=10),
            name="Best Path Destination",
        )
    )

    fig.update_layout(
        scene=dict(
            xaxis_title="X",
            yaxis_title="Y",
            zaxis_title="Z",
        ),
        title=f"Monte Carlo Paths in 4D ({dimension_interpretation})",
        showlegend=True,
        width=1200,
        height=800,
    )

    fig.show()


# Executar simulações e visualizar para diferentes abordagens
dimension_interpretations = [
    "4_spatial",
    "2_spatial_2_temporal",
    "3_temporal_1_spatial",
]
for interpretation in dimension_interpretations:
    paths, best_path = monte_carlo_4d(
        num_simulations=100, time_steps=50, dimension_interpretation=interpretation
    )
    visualize_paths(paths, best_path, interpretation)


# Hipersuperfícies em 4D
Vamos definir funções que mapeiam pontos 4D para valores 3D, 2D ou 1D, criando visualizações correspondentes

In [ ]:
import numpy as np
import plotly.graph_objs as go


# Definir a função matemática em 4D
def f_4d(x, y, z, w):
    return np.sin(x) * np.cos(y) + np.sin(z) * np.cos(w)


# Algoritmo de Monte Carlo para encontrar o melhor caminho
def monte_carlo_4d(num_simulations, time_steps, dimension_interpretation):
    paths = []
    best_path = None
    best_value = float("inf")

    for _ in range(num_simulations):
        path = np.cumsum(np.random.randn(time_steps, 4), axis=0)

        if dimension_interpretation == "4_spatial":
            value = np.sum(f_4d(path[:, 0], path[:, 1], path[:, 2], path[:, 3]))
        elif dimension_interpretation == "2_spatial_2_temporal":
            path[:, 2] = np.arange(time_steps)  # t1 as sequential time steps
            path[:, 3] = np.arange(time_steps)  # t2 as sequential time steps
            value = np.sum(f_4d(path[:, 0], path[:, 1], path[:, 2], path[:, 3]))
        elif dimension_interpretation == "3_temporal_1_spatial":
            path[:, 0] = np.arange(time_steps)  # t1 as sequential time steps
            path[:, 1] = np.arange(time_steps)  # t2 as sequential time steps
            path[:, 2] = np.arange(time_steps)  # t3 as sequential time steps
            value = np.sum(f_4d(path[:, 0], path[:, 1], path[:, 2], path[:, 3]))

        if value < best_value:
            best_value = value
            best_path = path

        paths.append(path)

    return paths, best_path


# Visualização com Plotly para diferentes abordagens
def visualize_paths(paths, best_path, dimension_interpretation, projection):
    fig = go.Figure()

    for path in paths:
        if projection == "4D_to_3D":
            fig.add_trace(
                go.Scatter3d(
                    x=path[:, 0],
                    y=path[:, 1],
                    z=path[:, 2],
                    mode="lines",
                    line=dict(width=2),
                    opacity=0.1,
                )
            )
        elif projection == "4D_to_2D":
            fig.add_trace(
                go.Scatter(
                    x=path[:, 0],
                    y=path[:, 1],
                    mode="lines",
                    line=dict(width=2),
                    opacity=0.1,
                )
            )
        elif projection == "4D_to_1D":
            fig.add_trace(
                go.Scatter(
                    x=np.arange(path.shape[0]),
                    y=path[:, 0],
                    mode="lines",
                    line=dict(width=2),
                    opacity=0.1,
                )
            )

    if projection == "4D_to_3D":
        fig.add_trace(
            go.Scatter3d(
                x=best_path[:, 0],
                y=best_path[:, 1],
                z=best_path[:, 2],
                mode="lines",
                line=dict(color="red", width=4),
                name="Best Path",
            )
        )

        fig.add_trace(
            go.Scatter3d(
                x=[best_path[0, 0]],
                y=[best_path[0, 1]],
                z=[best_path[0, 2]],
                mode="markers",
                marker=dict(color="green", size=10),
                name="Origin",
            )
        )

        fig.add_trace(
            go.Scatter3d(
                x=[best_path[-1, 0]],
                y=[best_path[-1, 1]],
                z=[best_path[-1, 2]],
                mode="markers",
                marker=dict(color="blue", size=10),
                name="Best Path Destination",
            )
        )

        fig.update_layout(
            scene=dict(
                xaxis_title="X",
                yaxis_title="Y",
                zaxis_title="Z",
            ),
            title=f"Monte Carlo Paths in 4D to 3D Projection ({dimension_interpretation})",
            showlegend=True,
            width=1200,
            height=800,
        )
    elif projection == "4D_to_2D":
        fig.add_trace(
            go.Scatter(
                x=best_path[:, 0],
                y=best_path[:, 1],
                mode="lines",
                line=dict(color="red", width=4),
                name="Best Path",
            )
        )

        fig.add_trace(
            go.Scatter(
                x=[best_path[0, 0]],
                y=[best_path[0, 1]],
                mode="markers",
                marker=dict(color="green", size=10),
                name="Origin",
            )
        )

        fig.add_trace(
            go.Scatter(
                x=[best_path[-1, 0]],
                y=[best_path[-1, 1]],
                mode="markers",
                marker=dict(color="blue", size=10),
                name="Best Path Destination",
            )
        )

        fig.update_layout(
            xaxis_title="X",
            yaxis_title="Y",
            title=f"Monte Carlo Paths in 4D to 2D Projection ({dimension_interpretation})",
            showlegend=True,
            width=1200,
            height=800,
        )
    elif projection == "4D_to_1D":
        fig.add_trace(
            go.Scatter(
                x=np.arange(best_path.shape[0]),
                y=best_path[:, 0],
                mode="lines",
                line=dict(color="red", width=4),
                name="Best Path",
            )
        )

        fig.add_trace(
            go.Scatter(
                x=[0],
                y=[best_path[0, 0]],
                mode="markers",
                marker=dict(color="green", size=10),
                name="Origin",
            )
        )

        fig.add_trace(
            go.Scatter(
                x=[best_path.shape[0] - 1],
                y=[best_path[-1, 0]],
                mode="markers",
                marker=dict(color="blue", size=10),
                name="Best Path Destination",
            )
        )

        fig.update_layout(
            xaxis_title="Step",
            yaxis_title="X",
            title=f"Monte Carlo Paths in 4D to 1D Projection ({dimension_interpretation})",
            showlegend=True,
            width=1200,
            height=800,
        )

    fig.show()


# Executar simulações e visualizar para diferentes abordagens
dimension_interpretations = [
    "4_spatial",
    "2_spatial_2_temporal",
    "3_temporal_1_spatial",
]
projections = ["4D_to_3D", "4D_to_2D", "4D_to_1D"]

for interpretation in dimension_interpretations:
    for projection in projections:
        paths, best_path = monte_carlo_4d(
            num_simulations=100, time_steps=50, dimension_interpretation=interpretation
        )
        visualize_paths(paths, best_path, interpretation, projection)


# Outra forma de representar essa hipersuperficie utilizando o monte carlo

In [ ]:
import numpy as np
import plotly.graph_objs as go


# Definir a função matemática em 4D
def f_4d(x, y, z, w):
    return np.sin(x) * np.cos(y) + np.sin(z) * np.cos(w)


# Algoritmo de Monte Carlo para encontrar o melhor caminho
def monte_carlo_4d(num_simulations, time_steps, dimension_interpretation):
    paths = []
    best_path = None
    best_value = float("inf")

    for _ in range(num_simulations):
        path = np.cumsum(np.random.randn(time_steps, 4), axis=0)

        if dimension_interpretation == "2_spatial_2_temporal":
            path[:, 2] = np.linspace(0, 1, time_steps)  # t1 as sequential time steps
            path[:, 3] = np.linspace(0, 1, time_steps)  # t2 as sequential time steps
        elif dimension_interpretation == "3_temporal_1_spatial":
            path[:, 0] = np.linspace(0, 1, time_steps)  # t1 as sequential time steps
            path[:, 1] = np.linspace(0, 1, time_steps)  # t2 as sequential time steps
            path[:, 2] = np.linspace(0, 1, time_steps)  # t3 as sequential time steps

        value = np.sum(f_4d(path[:, 0], path[:, 1], path[:, 2], path[:, 3]))

        if value < best_value:
            best_value = value
            best_path = path

        paths.append(path)

    return paths, best_path


# Função para criar a hipersuperfície
def create_hypersurface(dimension_interpretation):
    x = np.linspace(-5, 5, 30)
    y = np.linspace(-5, 5, 30)
    z = np.linspace(-5, 5, 30)
    w = np.linspace(-5, 5, 30)
    x, y, z, w = np.meshgrid(x, y, z, w)

    values = f_4d(x, y, z, w)

    if dimension_interpretation == "4_spatial":
        return x, y, z, values
    elif dimension_interpretation == "2_spatial_2_temporal":
        return x, y, w, values
    elif dimension_interpretation == "3_temporal_1_spatial":
        return y, z, w, values


# Visualização com Plotly para diferentes abordagens
def visualize_hypersurface_and_paths(dimension_interpretation, paths, best_path):
    x, y, z, values = create_hypersurface(dimension_interpretation)

    fig = go.Figure(
        data=go.Volume(
            x=x.flatten(),
            y=y.flatten(),
            z=z.flatten(),
            value=values.flatten(),
            isomin=np.min(values),
            isomax=np.max(values),
            opacity=0.1,  # Needs to be small to see through all surfaces
            surface_count=17,  # Number of isosurfaces, 2 by default
        )
    )

    # Adicionar todos os caminhos ao gráfico
    for path in paths:
        fig.add_trace(
            go.Scatter3d(
                x=path[:, 0],
                y=path[:, 1],
                z=path[:, 2],
                mode="lines+markers",
                line=dict(width=2),
                marker=dict(size=3),
                opacity=0.1,
            )
        )

    # Adicionar o melhor caminho em destaque
    fig.add_trace(
        go.Scatter3d(
            x=best_path[:, 0],
            y=best_path[:, 1],
            z=best_path[:, 2],
            mode="lines+markers",
            line=dict(color="red", width=4),
            marker=dict(size=5),
            name="Best Path",
        )
    )

    # Adicionar ponto de origem
    fig.add_trace(
        go.Scatter3d(
            x=[0],
            y=[0],
            z=[0],
            mode="markers",
            marker=dict(color="green", size=10),
            name="Origin",
        )
    )

    # Adicionar ponto de destino para o melhor caminho
    fig.add_trace(
        go.Scatter3d(
            x=[best_path[-1, 0]],
            y=[best_path[-1, 1]],
            z=[best_path[-1, 2]],
            mode="markers",
            marker=dict(color="blue", size=10),
            name="Best Path Destination",
        )
    )

    fig.update_layout(
        scene=dict(
            xaxis_title="X",
            yaxis_title="Y",
            zaxis_title="Z",
        ),
        title=f"Hypersurface in 4D ({dimension_interpretation})",
        width=1200,
        height=800,
    )

    fig.show()


# Executar simulações e visualizar para diferentes abordagens
dimension_interpretations = [
    "4_spatial",
    "2_spatial_2_temporal",
    "3_temporal_1_spatial",
]
for interpretation in dimension_interpretations:
    paths, best_path = monte_carlo_4d(
        num_simulations=100, time_steps=50, dimension_interpretation=interpretation
    )
    visualize_hypersurface_and_paths(interpretation, paths, best_path)


In [ ]:
import numpy as np
import plotly.graph_objs as go


# Definir a função matemática em 4D
def f_4d(x, y, z, w):
    return np.sin(x) * np.cos(y) + np.sin(z) * np.cos(w)


# Algoritmo de Monte Carlo para encontrar o melhor caminho
def monte_carlo_4d(num_simulations, time_steps, dimension_interpretation):
    paths = []
    best_path = None
    best_value = float("inf")

    for _ in range(num_simulations):
        path = np.cumsum(np.random.randn(time_steps, 6), axis=0)

        if dimension_interpretation == "2_spatial_2_temporal":
            path[:, 2] = np.linspace(0, 1, time_steps)  # t1 as sequential time steps
            path[:, 3] = np.linspace(0, 1, time_steps)  # t2 as sequential time steps
        elif dimension_interpretation == "3_temporal_1_spatial":
            path[:, 0] = np.linspace(0, 1, time_steps)  # t1 as sequential time steps
            path[:, 1] = np.linspace(0, 1, time_steps)  # t2 as sequential time steps
            path[:, 2] = np.linspace(0, 1, time_steps)  # t3 as sequential time steps
        elif dimension_interpretation == "3_temporal_2_spatial":
            path[:, 0] = np.linspace(0, 1, time_steps)  # t1 as sequential time steps
            path[:, 1] = np.linspace(0, 1, time_steps)  # t2 as sequential time steps
            path[:, 2] = np.linspace(0, 1, time_steps)  # t3 as sequential time steps
            path[:, 3] = path[:, 3]  # x
            path[:, 4] = path[:, 4]  # y
        elif dimension_interpretation == "3_temporal_3_spatial":
            path[:, 0] = np.linspace(0, 1, time_steps)  # t1 as sequential time steps
            path[:, 1] = np.linspace(0, 1, time_steps)  # t2 as sequential time steps
            path[:, 2] = np.linspace(0, 1, time_steps)  # t3 as sequential time steps
            path[:, 3] = path[:, 3]  # x
            path[:, 4] = path[:, 4]  # y
            path[:, 5] = path[:, 5]  # z
        elif dimension_interpretation == "2_temporal_3_spatial":
            path[:, 0] = np.linspace(0, 1, time_steps)  # t1 as sequential time steps
            path[:, 1] = np.linspace(0, 1, time_steps)  # t2 as sequential time steps
            path[:, 2] = path[:, 2]  # x
            path[:, 3] = path[:, 3]  # y
            path[:, 4] = path[:, 4]  # z

        value = np.sum(f_4d(path[:, 0], path[:, 1], path[:, 2], path[:, 3]))

        if value < best_value:
            best_value = value
            best_path = path

        paths.append(path)

    return paths, best_path


# Função para criar a hipersuperfície
def create_hypersurface(dimension_interpretation):
    x = np.linspace(-5, 5, 30)
    y = np.linspace(-5, 5, 30)
    z = np.linspace(-5, 5, 30)
    w = np.linspace(-5, 5, 30)
    x, y, z, w = np.meshgrid(x, y, z, w)

    values = f_4d(x, y, z, w)

    if dimension_interpretation == "4_spatial":
        return x, y, z, values
    elif dimension_interpretation == "2_spatial_2_temporal":
        return x, y, w, values
    elif dimension_interpretation == "3_temporal_1_spatial":
        return y, z, w, values
    elif dimension_interpretation == "3_temporal_2_spatial":
        return x, y, z, values
    elif dimension_interpretation == "3_temporal_3_spatial":
        return x, y, z, values
    elif dimension_interpretation == "2_temporal_3_spatial":
        return x, y, z, values


# Visualização com Plotly para diferentes abordagens
def visualize_hypersurface_and_paths(dimension_interpretation, paths, best_path):
    x, y, z, values = create_hypersurface(dimension_interpretation)

    fig = go.Figure(
        data=go.Volume(
            x=x.flatten(),
            y=y.flatten(),
            z=z.flatten(),
            value=values.flatten(),
            isomin=np.min(values),
            isomax=np.max(values),
            opacity=0.1,  # Needs to be small to see through all surfaces
            surface_count=17,  # Number of isosurfaces, 2 by default
        )
    )

    # Adicionar todos os caminhos ao gráfico
    for path in paths:
        fig.add_trace(
            go.Scatter3d(
                x=path[:, 0],
                y=path[:, 1],
                z=path[:, 2],
                mode="lines+markers",
                line=dict(width=2),
                marker=dict(size=3),
                opacity=0.1,
            )
        )

    # Adicionar o melhor caminho em destaque
    fig.add_trace(
        go.Scatter3d(
            x=best_path[:, 0],
            y=best_path[:, 1],
            z=best_path[:, 2],
            mode="lines+markers",
            line=dict(color="red", width=4),
            marker=dict(size=5),
            name="Best Path",
        )
    )

    # Adicionar ponto de origem
    fig.add_trace(
        go.Scatter3d(
            x=[0],
            y=[0],
            z=[0],
            mode="markers",
            marker=dict(color="green", size=10),
            name="Origin",
        )
    )

    # Adicionar ponto de destino para o melhor caminho
    fig.add_trace(
        go.Scatter3d(
            x=[best_path[-1, 0]],
            y=[best_path[-1, 1]],
            z=[best_path[-1, 2]],
            mode="markers",
            marker=dict(color="blue", size=10),
            name="Best Path Destination",
        )
    )

    fig.update_layout(
        scene=dict(
            xaxis_title="X",
            yaxis_title="Y",
            zaxis_title="Z",
        ),
        title=f"Hypersurface in 4D ({dimension_interpretation})",
        width=1200,
        height=800,
    )

    fig.show()


# Executar simulações e visualizar para diferentes abordagens
dimension_interpretations = [
    "4_spatial",
    "2_spatial_2_temporal",
    "3_temporal_1_spatial",
    "3_temporal_2_spatial",
    "3_temporal_3_spatial",
    "2_temporal_3_spatial",
]
for interpretation in dimension_interpretations:
    paths, best_path = monte_carlo_4d(
        num_simulations=100, time_steps=50, dimension_interpretation=interpretation
    )
    visualize_hypersurface_and_paths(interpretation, paths, best_path)
